### Initialize variables

In [ ]:
lang = 'en'
sg = 1
#architecture = 'mlp'
#architecture = 'cnn'
architecture = 'lstm'
#vector = 'word2vec_skipgram'
#vector = 'word2vec_cbow'
#vector = 'spacy'
vector = 'tfkerastokenizer'
#vector = 'tfidf_hashing_vectorizer'
optimizer_name = 'sgd'
#optimizer_name = 'adam'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd '/content/drive/MyDrive/Colab Notebooks/final/enron_sent/'

import os
global PATH
PATH = os.getcwd() + '/'

!pip install import-ipynb -q
import import_ipynb

import include as i
spacy_path = i.spacy_paths[lang]
stopwords_lang = i.stopwords_langs[lang]

/content/drive/MyDrive/Colab Notebooks/final/enron_sent
     |████████████████████████████████| 1.6 MB 11.7 MB/s 
importing Jupyter notebook from include.ipynb


In [ ]:
#import subprocess
#for copy_lang in i.copy_langs:
#  command = "cp enron_sent_en.ipynb enron_sent_" + copy_lang + ".ipynb"
#  print(command)
#  process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
#  output, error = process.communicate()

cp enron_sent_en.ipynb enron_sent_fi.ipynb
cp enron_sent_en.ipynb enron_sent_sv.ipynb
cp enron_sent_en.ipynb enron_sent_no.ipynb
cp enron_sent_en.ipynb enron_sent_de.ipynb


### Initialize libraries

In [ ]:
import base64
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns
import string
import tensorflow as tf
import time

import nltk
nltk.download('stopwords')

import spacy

from collections import Counter
from keras.layers import Dense, Dropout, Input, LSTM, Embedding, Bidirectional, Flatten, SpatialDropout1D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential, Model
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import log_loss
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Accuracy,Precision,Recall,AUC,CategoricalAccuracy,CategoricalCrossentropy,CategoricalHinge
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

#https://stackoverflow.com/questions/49284455/keras-custom-function-implementing-jaccard
def jaccard_score(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac/100 * smooth

### Initialize language

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words(stopwords_lang)

In [ ]:
nlp = spacy.load(PATH + lang + spacy_path)

### Load data

In [ ]:
%%time
df_orig = pd.read_csv(PATH + 'data/users_as_labels_sent.csv')
# Use pickle to save space
#df_orig = pickle.load( open( PATH + 'data/df_' + lang, 'rb' ) ) # Preprocessed df
#df_orig = pickle.load( open( PATH + lang + '/data/df_' + lang, 'rb' ) ) # Ready df

CPU times: user 76 ms, sys: 62 ms, total: 138 ms
Wall time: 157 ms


In [ ]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39624 entries, 0 to 39623
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    39624 non-null  object
 1   user    39624 non-null  object
 2   folder  39624 non-null  object
dtypes: object(3)
memory usage: 928.8+ KB


In [ ]:
df_orig.head()

,text,user,folder
0,\n I don't know if you got these.\n ----------...,bass-e,sent
1,\n Can you spell S-N-O-O-T-Y?\n \n e\n \n \n ...,bass-e,sent
2,"\n The new deal #s are 147888,147889.\n \n -E",bass-e,sent
3,\n I think I sent this to the worng e-mail add...,bass-e,sent
4,\n We are going to Woody's tonight if you are ...,bass-e,sent


In [ ]:
df_orig.user.value_counts()

mann-k          8926
kaminski-v      8644
dasovich-j      5366
germany-c       5128
shackleton-s    4407
jones-t         4123
bass-e          3030
Name: user, dtype: int64

In [ ]:
df = df_orig.copy()
df['lang'] = lang
df['length'] = df.text.apply(lambda x: len(x))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39624 entries, 0 to 39623
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    39624 non-null  object
 1   user    39624 non-null  object
 2   folder  39624 non-null  object
 3   lang    39624 non-null  object
 4   length  39624 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [ ]:
df.head()

,text,user,folder,lang,length
0,\n I don't know if you got these.\n ----------...,bass-e,sent,en,545
1,\n Can you spell S-N-O-O-T-Y?\n \n e\n \n \n ...,bass-e,sent,en,460
2,"\n The new deal #s are 147888,147889.\n \n -E",bass-e,sent,en,42
3,\n I think I sent this to the worng e-mail add...,bass-e,sent,en,167
4,\n We are going to Woody's tonight if you are ...,bass-e,sent,en,64


### Translate

#### Do translate

In [ ]:
#!pip install googletrans==3.1.0a0 -q

In [ ]:
#from googletrans import Translator
#translator = Translator()

In [ ]:
#Find if some features are too long for Google to translate
#df.length.value_counts().sort_index(ascending=False)

In [ ]:
#%%time
#translations = []
#for key in df.index:
#  if key == 3430:
#    line = line[:145900] # Experimentally established that max character limit Google can translate is ~145900
#  else:
#    line = df.text[key]
#  translations.append(translator.translate(line, src = 'en', dest=lang).text)

In [ ]:
#df['translation'] = translations
df['translation'] = df.text # No need, because language already is EN

#### Save translate

In [ ]:
# Use pickle to save space
pickle.dump( df, open( '/content/drive/MyDrive/Colab Notebooks/final/rtu/' + lang + '/data/df_' + lang, 'wb' ) )

#### Load translate

In [ ]:
df = pickle.load(open( '/content/drive/MyDrive/Colab Notebooks/final/rtu/' + lang + '/data/df_' + lang, 'rb'))

###Spacy

In [ ]:
#https://www.kaggle.com/code/enerrio/scary-nlp-with-spacy-and-keras/notebook

In [ ]:
# Clean text before feeding it to Spacy
punctuations = string.punctuation
from collections import defaultdict

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [ ]:
%%time
# Cleanup text and make sure it retains original shape
print('Original training data shape: ', df.translation.shape)
train_cleaned = cleanup_text(df.translation, logging=True)
print('Cleaned up training data shape: ', train_cleaned.shape)

In [ ]:
pickle.dump(train_cleaned, open(PATH + lang + '/texts/spacy_' + lang, 'wb'))

In [ ]:
train_cleaned = pickle.load(open(PATH + lang + '/texts/spacy_' + lang, 'rb'))

In [ ]:
df['translation_clean'] = train_cleaned.values

In [ ]:
df.translation_clean.isna().sum()

In [ ]:
df.translation_clean.fillna(' ', inplace=True)

In [ ]:
# Use pickle to save space
pickle.dump( df, open( PATH + lang + '/data/df_' + lang, 'wb' ) )

In [ ]:
# Use pickle to save space
df = pickle.load( open( PATH + lang + '/data/df_' + lang, 'rb' ) )

In [ ]:
%%time
# Parse documents and print some info
print('Parsing documents...')

train_vec = []
c = 0
for doc in nlp.pipe(df.translation_clean, batch_size=500):
    if doc.has_vector:
        train_vec.append(doc.vector)
    # If doc doesn't have a vector, then fill it with zeros.
    else:
        train_vec.append(np.zeros((300,), dtype="float32"))
    if c%100 == 0: print(f'c: {c}')
    c+=1
        
# train_vec = [doc.vector for doc in nlp.pipe(train_cleaned, batch_size=500)]
train_vec = np.array(train_vec)

print('Total number of documents parsed: {}'.format(len(train_vec)))
print('Number of words in first document: ', len(df.translation_clean.iloc[0]))
print('Number of words in second document: ', len(df.translation_clean.iloc[1]))

In [ ]:
train_vec.shape

In [ ]:
train_vec[0]

In [ ]:
pickle.dump(train_vec,open(PATH + lang + '/vectors/spacy_' + lang, 'wb'))

In [ ]:
train_vec = pickle.load(open(PATH + lang + '/vectors/spacy_' + lang, 'rb'))

###Word2vec

In [ ]:
all_text = pd.DataFrame({'text_clean':df.text_clean})
print('Number of total text documents:', len(all_text))

In [ ]:
# Define function to preprocess text for a word2vec model
def cleanup_text_word2vec(docs, logging=False):
    sentences = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents" % (counter, len(docs)))
        # Disable tagger so that lemma_ of personal pronouns (I, me, etc) don't getted marked as "-PRON-"
        doc = nlp(doc, disable=['tagger'])
        # Grab lemmatized form of words and make lowercase
        doc = " ".join([tok.lemma_.lower() for tok in doc])
        # Split into sentences based on punctuation
        doc = re.split("[\.?!;] ", doc)
        # Remove commas, periods, and other punctuation (mostly commas)
        doc = [re.sub("[\.,;:!?]", "", sent) for sent in doc]
        # Split into words
        doc = [sent.split() for sent in doc]
        sentences += doc
        counter += 1
    return sentences

In [ ]:
%%time
train_cleaned_word2vec = cleanup_text_word2vec(all_text.text_clean, logging=True)
print('Cleaned up training data size (i.e. number of sentences): ', len(train_cleaned_word2vec))

In [ ]:
pickle.dump(train_cleaned_word2vec,open(PATH + lang + '/texts/word2vec_' + lang, 'wb'))

In [ ]:
train_cleaned_word2vec = pickle.load(open(PATH + lang + '/texts/word2vec_' + lang, 'rb'))

In [ ]:
train_cleaned_word2vec[0]

In [ ]:
!pip install gensim -U -q

In [ ]:
import gensim

In [ ]:
gensim.__version__

In [ ]:
%%time
from gensim.models.word2vec import Word2Vec

text_dim = 300
print("Training Word2Vec model...")
wordvec_model = Word2Vec(train_cleaned_word2vec, vector_size=text_dim, window=5, min_count=3, workers=4, sg=sg)
print("Word2Vec model created.")

Training Word2Vec model...
Word2Vec model created.
CPU times: user 4min 8s, sys: 783 ms, total: 4min 9s
Wall time: 1min 25s


In [ ]:
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.key_to_index), text_dim))

In [ ]:
pickle.dump(wordvec_model,open(PATH + lang + '/models/vector/' + vector + '_30k_300d_' + lang, 'wb'))

In [ ]:
wordvec_model = pickle.load(open(PATH + lang + '/models/vector/' + vector + '_30k_300d_' + lang, 'rb'))

In [ ]:
print(wordvec_model.wv.most_similar(positive=['presentation', 'scheduled'], negative=['market']))
print(wordvec_model.wv.most_similar_cosmul(positive=['presentation', 'scheduled'], negative=['market']))
print(wordvec_model.wv.doesnt_match("Please read the following".split()))
print(wordvec_model.wv.similarity('presentation', 'scheduled'))

In [ ]:
# Define function to create word vectors given a cleaned piece of text.
def create_average_vec(doc):
    average = np.zeros((text_dim,), dtype='float32')
    num_words = 0.
    for word in doc.split():
        if word in wordvec_model.wv.key_to_index:
            average = np.add(average, wordvec_model.wv.get_vector(word))
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [ ]:
# Counting the number of empty strings are in df.translation_clean
count = 0
for i in range(len(df.translation_clean)):
    if df.translation_clean.iloc[i] == "":
        print("index:", i)
        count += 1
print(count)

In [ ]:
%%time
# Create word vectors
text_dim = 300
train_cleaned_vec = np.zeros((df.translation_clean.shape[0], text_dim), dtype="float32")  # 19579 x 300
df.translation_clean.fillna('', inplace=True)
for i in range(len(df.translation_clean)):
    train_cleaned_vec[i] = create_average_vec(df.translation_clean.iloc[i])

print("Train word vector shape:", train_cleaned_vec.shape)

In [ ]:
pickle.dump(train_cleaned_vec, open(PATH + lang + '/vectors/' + vector + '_30k_300d_' + lang, 'wb'))

In [ ]:
train_cleaned_vec = pickle.load(open(PATH + lang + '/vectors/' + vector + '_30k_300d_' + lang, 'rb'))

###Define X and y

####Spacy

In [ ]:
# Spacy
X = train_vec
#X = normalize(train_vec, norm='max') # Sometimes returns better results if vecors are normalized
y = pd.get_dummies(df.user).values

In [ ]:
X[0]

array([ 0.8429706 , -0.26197803,  0.21154654, -0.29519996,  0.02114503,
        0.02414417, -0.30032158, -0.05977635,  0.3198796 , -0.323849  ,
        0.32592413,  0.46902856, -0.15496871, -0.5672165 ,  0.04649749,
        0.1975309 ,  0.60665774,  0.04762324,  0.4248602 , -0.11449067,
       -0.07872303, -0.71127105, -0.03501398,  0.3654025 ,  0.2354873 ,
       -0.07174928, -0.23417112,  0.2824742 ,  0.14211388, -0.08474524,
        0.22649744,  0.3058503 , -0.5110276 , -0.46810082,  0.5561856 ,
       -0.12731288, -0.48204336,  0.82135403, -0.34464285, -0.15282296,
       -0.15688428,  0.22655237, -0.05808501, -0.92754954,  0.7238813 ,
       -0.20188017,  0.47174   ,  0.22790341,  0.0096606 , -0.7248446 ,
        0.5777744 ,  0.21609631, -0.16893342, -0.5189102 , -0.10995514,
        0.3476664 ,  0.5372183 , -0.29847115, -0.36330727, -0.02420609,
       -0.29751977, -0.34877685,  0.4767719 , -0.85584396,  0.00923444,
       -0.58088946, -0.34584993, -0.07175385,  0.21119946, -0.83

####Word2Vec

In [ ]:
# Word2Vec
X = train_cleaned_vec
y = pd.get_dummies(df.user).values

In [ ]:
X.shape

(39624, 300)

####Tokenizer

In [ ]:
def get_sequences(texts, tokenizer, train=True, max_seq_length=None):
    sequences = tokenizer.texts_to_sequences(texts)
    
    if train == True:
        max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
        print('max_seq_length: '+str(max_seq_length))
    
    sequences = pad_sequences(sequences, maxlen=300, padding='post')
    
    return sequences

In [ ]:
%%time
tokenizer = Tokenizer(num_words=30000, lower=True)
tokenizer.fit_on_texts(df.translation_clean.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X = get_sequences(df.translation_clean.values, tokenizer, train=True)
y = np.array(pd.get_dummies(df.user))

Found 82820 unique tokens.
max_seq_length: 17157
CPU times: user 8.4 s, sys: 115 ms, total: 8.52 s
Wall time: 8.71 s


####Hashingvectorizer

In [ ]:
%%time
from sklearn.feature_extraction.text import HashingVectorizer
features_hash = HashingVectorizer(ngram_range=(1,2),n_features=300).transform(df.translation_clean)
features_matrix = features_hash

CPU times: user 6.85 s, sys: 56.6 ms, total: 6.91 s
Wall time: 6.95 s


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(features_matrix)

In [ ]:
%%time
X = tfidf_transformer.transform(features_matrix).toarray()
y = pd.get_dummies(df.user).values

CPU times: user 94.5 ms, sys: 157 ms, total: 252 ms
Wall time: 254 ms


In [ ]:
X.shape

(39624, 300)

###Define model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('X_train size: {}'.format(X_train.shape))
print('X_test size: {}'.format(X_test.shape))
print('y_train size: {}'.format(y_train.shape))
print('y_test size: {}'.format(y_test.shape))

X_train size: (27736, 300)
X_test size: (11888, 300)
y_train size: (27736, 7)
y_test size: (11888, 7)


In [ ]:
def build_model(architecture='mlp'):
    model = Sequential()
    if architecture == 'mlp':
        # Densely Connected Neural Network (Multi-Layer Perceptron)
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal', input_dim=X_train[0].shape[0]))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        #model.add(Dense(3, activation='softmax'))
        model.add(Dense(y_train.shape[1], activation='softmax'))
    elif architecture == 'cnn':
        # 1-D Convolutional Neural Network
        inputs = Input(shape=(X_train[0].shape[0],1))

        x = Conv1D(64, 3, strides=1, padding='same', activation='relu')(inputs)

        #Cuts the size of the output in half, maxing over every 2 inputs
        x = MaxPooling1D(pool_size=2)(x)
        x = Conv1D(128, 3, strides=1, padding='same', activation='relu')(x)
        x = GlobalMaxPooling1D()(x) 
        outputs = Dense(y_train.shape[1], activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs, name='CNN')
    elif architecture == 'lstm':
        # LSTM network
        inputs = Input(shape=(X_train[0].shape[0],))
        embedding = Embedding(input_dim=30000,output_dim=64)(inputs)
        #lstm1 = Bidirectional(LSTM(256,input_length=X_train.shape[1],return_sequences=True,dropout=0.2,recurrent_dropout=0,activation="tanh",recurrent_activation="sigmoid",unroll=False,use_bias=True))(embedding)
        lstm1 = LSTM(256,input_length=X_train.shape[1],return_sequences=True,dropout=0.2,recurrent_dropout=0,activation="tanh",recurrent_activation="sigmoid",unroll=False,use_bias=True)(embedding)
        #lstm2 = tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0, activation="tanh",recurrent_activation="sigmoid",unroll=False,use_bias=True)(lstm1)
        flatten = Flatten()(lstm1)
        #outputs = Dense(y_train.shape[1], activation='sigmoid')(flatten)
        outputs = Dense(y_train.shape[1], activation='softmax')(flatten)
        model = Model(inputs=inputs, outputs=outputs)

        #model = Sequential()
        #model.add(Embedding(30000, 64, input_length=X_train.shape[1]))
        #model.add(SpatialDropout1D(0.2))
        #model.add(LSTM(256,input_length=X_train.shape[1],return_sequences=False,dropout=0.2,recurrent_dropout=0,activation="tanh",recurrent_activation="sigmoid",unroll=False,use_bias=True))
        #model.add(LSTM(256, dropout=0.2, recurrent_dropout=0, activation="tanh",recurrent_activation="sigmoid",unroll=False,use_bias=True))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
    else:
        print('Error: Model type not found.')
    return model

In [ ]:
# Define keras model
# Using MLP in kernel for speed
#model = build_model('mlp')
#model = build_model('cnn')
#model = build_model('lstm')
model = build_model(architecture)

# If the model is a CNN then expand the dimensions of the training data
#if model.name == "CNN" or model.name == "LSTM":
if model.name == "CNN":
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    print('Text train shape: ', X_test.shape)
    print('Text test shape: ', X_test.shape)
    
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 300)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 300, 64)           1920000   
                                                                 
 lstm_2 (LSTM)               (None, 300, 256)          328704    
                                                                 
 flatten_2 (Flatten)         (None, 76800)             0         
                                                                 
 dense_54 (Dense)            (None, 7)                 537607    
                                                                 
Total params: 2,786,311
Trainable params: 2,786,311
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=[
                        'accuracy',
                        Precision(),
                        Recall(),
                        get_f1,
                        jaccard_score,
                        AUC(),
                        CategoricalAccuracy(),
                        CategoricalCrossentropy(),
                        CategoricalHinge()
                        ])
#Accuracy()

###Train model

In [ ]:
%%time
start_time = time.time()

# Define number of epochs
epochs = 200

# Fit the model to the training data
estimator = model.fit(X_train, y_train,
                      validation_split=0.2,
                      epochs=epochs,
                      batch_size=128,
                      verbose=1,
                      validation_data=(X_test, y_test),
                      callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,min_delta=0.001,verbose=1)]
                      )

execution_time = time.time() - start_time

Epoch 1/200
217/217 [==============================] - 514s 2s/step - loss: 1.8593 - accuracy: 0.2442 - precision_18: 0.0000e+00 - recall_18: 0.0000e+00 - get_f1: 0.0000e+00 - jaccard_score: 0.9837 - auc_18: 0.6372 - categorical_accuracy: 0.2442 - categorical_crossentropy: 1.8593 - categorical_hinge: 1.0615 - val_loss: 1.8321 - val_accuracy: 0.2902 - val_precision_18: 0.0000e+00 - val_recall_18: 0.0000e+00 - val_get_f1: 0.0000e+00 - val_jaccard_score: 0.9838 - val_auc_18: 0.6599 - val_categorical_accuracy: 0.2902 - val_categorical_crossentropy: 1.8321 - val_categorical_hinge: 1.0608
Epoch 2/200
217/217 [==============================] - 507s 2s/step - loss: 1.6471 - accuracy: 0.3748 - precision_18: 0.8691 - recall_18: 0.1078 - get_f1: 0.1714 - jaccard_score: 0.9853 - auc_18: 0.7376 - categorical_accuracy: 0.3748 - categorical_crossentropy: 1.6471 - categorical_hinge: 0.9823 - val_loss: 1.2204 - val_accuracy: 0.5479 - val_precision_18: 0.9775 - val_recall_18: 0.3109 - val_get_f1: 0.4701

In [ ]:
model.save(PATH + lang +'/models/ann/' + architecture + '_' + vector + '_30k_300d_' + optimizer_name + '_' + lang + '/')

In [ ]:
%%time
results = model.evaluate(X_test, y_test)
results[len(results) - 1] = execution_time

In [ ]:
pickle.dump(results,open(PATH + lang + '/results/' + architecture + '_' + vector + '_30k_300d_' + optimizer_name + '_' + lang, 'wb'))

In [ ]:
print("Test Loss: {:.2f}%".format(results[0] * 100))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print("Test Precision: {:.2f}%".format(results[2] * 100))
print("Test Recall: {:.2f}%".format(results[3] * 100))
print("Test F-1 Score: {:.2f}%".format(results[4] * 100))
print("Test Jaccard Score: {:.2f}%".format(results[5] * 100))
print("Test AUC ar ROC līkni: {:.2f}%".format(results[6] * 100))
print("Test Categorical Accuracy: {:.2f}%".format(results[7] * 100))
print("Test Categorical Crossentropy: {:.2f}%".format(results[8] * 100))
print("Test Categorical Hinge: {:.2f}%".format(results[9] * 100))
print("Execution time: {:f}".format(results[len(results)-1]))

Test Loss: 24.38%
Test Accuracy: 92.40%
Test Precision: 96.01%
Test Recall: 90.46%
Test F-1 Score: 93.09%
Test Jaccard Score: 99.81%
Test AUC ar ROC līkni: 99.27%
Test Categorical Accuracy: 92.40%
Test Categorical Crossentropy: 24.38%
Test Categorical Hinge: 935048.28%
Execution time: 9350.482825


In [ ]:
graph_data = pd.DataFrame(estimator.history)

In [ ]:
pickle.dump(graph_data,open(PATH + lang + '/graph_data/' + architecture + '_' + vector + '_30k_300d_' + optimizer_name + '_' + lang, 'wb'))

In [ ]:
%%time
plt.style.use('ggplot')
graph_data.plot()